<a href="https://colab.research.google.com/github/1kaiser/test2023/blob/main/TestingSlicingImagesSAHI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import pathlib
dir='/content/files'
urls = 'https://github.com/1kaiser/Snow-cover-area-estimation/releases/download/v1/imagesfolder.zip'
data_dir = tf.keras.utils.get_file(origin=urls,
                                   fname='s',
                                   cache_subdir= dir,
                                   archive_format='auto',
                                   untar=False,
                                   extract=True)
!rm -r {dir}/s
data_dir = pathlib.Path(data_dir)

1116364563/1116364563 [==============================] - 32s 0us/step


#head

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import re

from osgeo import gdal

def tif2array(input_file, calc_gain=True):

    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount), dtype=float)
   
    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        image[:, :, b] = band.ReadAsArray()
    return image, dataset


In [6]:
image_dir = r'/content/files/'

#############################################################################
prefix = "sur_refl_"
end = ["b01", "b02", "b03", "b04", "b05", "b06", "b07", "day_of_year", "qc_500m", "raz", "state_500m", "szen", "vzen"]
DayOY = "_doy\[0-9]+_aid0001"
fileExt = r'.tif'
expression_b1 = prefix+end[0]
expression_b2 = prefix+end[1]
expression_b3 = prefix+end[2]
expression_b4 = prefix+end[3]
expression_b5 = prefix+end[4]
expression_b6 = prefix+end[5]
expression_b7 = prefix+end[6]


imgs_list_b1 = [f for f in os.listdir(image_dir) if f.__contains__(expression_b1)]

imgs_list_b1.sort(reverse=True)       

In [7]:
from google.colab import output
temp_dir = r'/content/'

def ybatchedimages(images_path, image_list, batch_idx):
  images = []
  path = os.path.join(images_path, image_list[batch_idx])
  pathb2 = path.replace(expression_b1, expression_b2)
  pathb4 = path.replace(expression_b1, expression_b4)
  pathb6 = path.replace(expression_b1, expression_b6)

  #creating file NDSI
  !gdal_calc.py \
    --overwrite \
    --type=Float32 \
    --NoDataValue=0 \
    -A {pathb4} \
    --A_band 1 \
    -B {pathb6} \
    --B_band 1 \
    -C {pathb2} \
    --C_band 1 \
    --outfile={temp_dir}"BothCheck_result_final.tif" \
    --calc="(((A.astype(float) - B)/(A.astype(float) + B))>=0.4)*(C.astype(float)/10000>0.11)"

  pathout = temp_dir+str('BothCheck_result_final.tif')
  images.append(tif2array(pathout, 0)[0])

  !rm -r {temp_dir}"BothCheck_result_final.tif"
  output.clear()
  return images


import jax.numpy as jnp
def xbatchedimages(images_path, image_list, batch_idx):
  images = []
  path = os.path.join(images_path, image_list[batch_idx])
  v1 = tif2array(path.replace(expression_b1, expression_b1),0)[0]
  v2 = jnp.append(v1, tif2array(path.replace(expression_b1, expression_b2),0)[0] , axis =2)
  v3 = jnp.append(v2, tif2array(path.replace(expression_b1, expression_b3),0)[0] , axis =2)
  v4 = jnp.append(v3, tif2array(path.replace(expression_b1, expression_b4),0)[0] , axis =2)
  v5 = jnp.append(v4, tif2array(path.replace(expression_b1, expression_b5),0)[0] , axis =2)
  v6 = jnp.append(v5, tif2array(path.replace(expression_b1, expression_b6),0)[0] , axis =2)
  v7 = jnp.append(v6, tif2array(path.replace(expression_b1, expression_b7),0)[0] , axis =2)
  images.append(v7)
  w1 = tif2array(path.replace(expression_b1, expression_b1),0)[0]
  w2 = tif2array(path.replace(expression_b1, expression_b2),0)[0]
  w3 = tif2array(path.replace(expression_b1, expression_b3),0)[0]
  w4 = tif2array(path.replace(expression_b1, expression_b4),0)[0]
  w5 = tif2array(path.replace(expression_b1, expression_b5),0)[0]
  w6 = tif2array(path.replace(expression_b1, expression_b6),0)[0]
  w7 = tif2array(path.replace(expression_b1, expression_b7),0)[0]
  return images

import jax.random as random
import jax.numpy as jnp
batch_size = 1
no_of_batches = int(len(imgs_list_b1)/batch_size)
def data_stream(i, no_of_batches):
  return jnp.asarray(xbatchedimages(image_dir, imgs_list_b1, i)), jnp.asarray(ybatchedimages(image_dir, imgs_list_b1, i))

In [100]:

def get_slice_bboxes( image_height: int, image_width: int, slice_height: int = 224 , slice_width: int = 224, overlap_height_ratio: float = 0.0, overlap_width_ratio: float = 0.0 ):
  # """Slices `image_pil` in crops.
  # Corner values of each slice will be generated using the `slice_height`,
  # `slice_width`, `overlap_height_ratio` and `overlap_width_ratio` arguments.
  # Args:
  #     image_height (int): Height of the original image.
  #     image_width (int): Width of the original image.
  #     slice_height (int): Height of each slice. Default 512.
  #     slice_width (int): Width of each slice. Default 512.
  #     overlap_height_ratio(float): Fractional overlap in height of each
  #         slice (e.g. an overlap of 0.2 for a slice of size 100 yields an
  #         overlap of 20 pixels). Default 0.2.
  #     overlap_width_ratio(float): Fractional overlap in width of each
  #         slice (e.g. an overlap of 0.2 for a slice of size 100 yields an
  #         overlap of 20 pixels). Default 0.2.
  #     auto_slice_resolution (bool): if not set slice parameters such as slice_height and slice_width,
  #         it enables automatically calculate these params from image resolution and orientation.
  # Returns:
  #     List[List[int]]: List of 4 corner coordinates for each N slices.
  #         [
  #             [slice_0_left, slice_0_top, slice_0_right, slice_0_bottom],
  #             ...
  #             [slice_N_left, slice_N_top, slice_N_right, slice_N_bottom]
  #         ]
  # """
  slice_bboxes = []
  y_max = y_min = 0

  if slice_height and slice_width:
      y_overlap = int(overlap_height_ratio * slice_height)
      x_overlap = int(overlap_width_ratio * slice_width)
  else:
      raise ValueError("Compute type is not auto and slice width and height are not provided.")

  while y_max < image_height:
      x_min = x_max = 0
      y_max = y_min + slice_height
      while x_max < image_width:
          x_max = x_min + slice_width
          if y_max > image_height or x_max > image_width:
              xmax = min(image_width, x_max)
              ymax = min(image_height, y_max)
              xmin = max(0, xmax - slice_width)
              ymin = max(0, ymax - slice_height)
              slice_bboxes.append([xmin, ymin, xmax, ymax])
          else:
              slice_bboxes.append([x_min, y_min, x_max, y_max])
          x_min = x_max - x_overlap
      y_min = y_max - y_overlap
  return slice_bboxes




In [3]:
slices = get_slice_bboxes(233, 454)
for slice in slices:
  print(slice)
  

[0, 0, 224, 224]
[224, 0, 448, 224]
[230, 0, 454, 224]
[0, 9, 224, 233]
[224, 9, 448, 233]
[230, 9, 454, 233]


In [101]:
b = data_stream(110, no_of_batches)
# print(b)

def slicedImageInput(i, b):
    b1 = b[0]
    b1.swapaxes(1,3).swapaxes(2,3)
    ######################<< xinput images sliced
    ximages = []
    for j in range(b1.shape[3]):
      if (j==0):
        a = b1[:, i[1]:i[3], i[0]:i[2], j]
      else:
        a = np.append(a, b1[:, i[1]:i[3], i[0]:i[2], j], axis = 0)
    ximages = np.expand_dims(a, axis = 0)
    ximages = ximages.swapaxes(1,2).swapaxes(2,3)

    b2 = b[1]
    b2.swapaxes(1,3).swapaxes(2,3)
    ######################<< yinput images sliced
    yimages = []
    for j in range(b2.shape[3]):
      if (j==0):
        a = b2[:, i[1]:i[3], i[0]:i[2], j]
      else:
        a = np.append(a, b2[:, i[1]:i[3], i[0]:i[2], j], axis = 0)
    yimages = np.expand_dims(a, axis = 0)
    yimages = yimages.swapaxes(1,2).swapaxes(2,3)
    return ximages, yimages

for i in get_slice_bboxes(b[0].shape[1], b[0].shape[2]):
  print(i, "<< th slice")
  slicedImageInput(i, b)

# slicedImageInput(get_slice_bboxes(b[0].shape[1], b[0].shape[2])[0], b)

[0, 0, 256, 233] << th slice
[198, 0, 454, 233] << th slice


In [35]:
get_slice_bboxes(b.shape[2], b.shape[3])
b[:,4,i[1]:i[3], i[0]:i[2]].shape

(1, 224, 224)

In [ ]:

def slice_image(
    image: Union[str, Image.Image],
    output_file_name: Optional[str] = None,
    output_dir: Optional[str] = None,
    out_ext: Optional[str] = None,
    verbose: bool = False,
) -> SliceImageResult:
    """Slice a large image into smaller windows. If output_file_name is given export
    sliced images.
    Args:
        image (str or PIL.Image): File path of image or Pillow Image to be sliced.
        coco_annotation_list (CocoAnnotation): List of CocoAnnotation objects.
        output_file_name (str, optional): Root name of output files (coordinates will
            be appended to this)
        output_dir (str, optional): Output directory
        slice_height (int): Height of each slice. Default 512.
        slice_width (int): Width of each slice. Default 512.
        overlap_height_ratio (float): Fractional overlap in height of each
            slice (e.g. an overlap of 0.2 for a slice of size 100 yields an
            overlap of 20 pixels). Default 0.2.
        overlap_width_ratio (float): Fractional overlap in width of each
            slice (e.g. an overlap of 0.2 for a slice of size 100 yields an
            overlap of 20 pixels). Default 0.2.
        auto_slice_resolution (bool): if not set slice parameters such as slice_height and slice_width,
            it enables automatically calculate these params from image resolution and orientation.
        min_area_ratio (float): If the cropped annotation area to original annotation
            ratio is smaller than this value, the annotation is filtered out. Default 0.1.
        out_ext (str, optional): Extension of saved images. Default is the
            original suffix.
        verbose (bool, optional): Switch to print relevant values to screen.
            Default 'False'.
    Returns:
        sliced_image_result: SliceImageResult:
                                sliced_image_list: list of SlicedImage
                                image_dir: str
                                    Directory of the sliced image exports.
                                original_image_size: list of int
                                    Size of the unsliced original image in [height, width]
        num_total_invalid_segmentation: int
            Number of invalid segmentation annotations.
    """

    # define verboseprint
    verboselog = logger.info if verbose else lambda *a, **k: None

    def _export_single_slice(image: np.ndarray, output_dir: str, slice_file_name: str):
        image_pil = read_image_as_pil(image)
        slice_file_path = str(Path(output_dir) / slice_file_name)
        # export sliced image
        image_pil.save(slice_file_path)
        image_pil.close()  # to fix https://github.com/obss/sahi/issues/565
        verboselog("sliced image path: " + slice_file_path)

    # create outdir if not present
    if output_dir is not None:
        Path(output_dir).mkdir(parents=True, exist_ok=True)

    # read image
    image_pil = read_image_as_pil(image)
    verboselog("image.shape: " + str(image_pil.size))

    image_width, image_height = image_pil.size
    if not (image_width != 0 and image_height != 0):
        raise RuntimeError(f"invalid image size: {image_pil.size} for 'slice_image'.")
    slice_bboxes = get_slice_bboxes(
        image_height=image_height,
        image_width=image_width,
    )

    t0 = time.time()
    n_ims = 0

    # init images and annotations lists
    sliced_image_result = SliceImageResult(original_image_size=[image_height, image_width], image_dir=output_dir)

    image_pil_arr = np.asarray(image_pil)
    # iterate over slices
    for slice_bbox in slice_bboxes:
        n_ims += 1

        # extract image
        tlx = slice_bbox[0]
        tly = slice_bbox[1]
        brx = slice_bbox[2]
        bry = slice_bbox[3]
        image_pil_slice = image_pil_arr[tly:bry, tlx:brx]

        # set image file suffixes
        slice_suffixes = "_".join(map(str, slice_bbox))
        if out_ext:
            suffix = out_ext
        else:
            try:
                suffix = Path(image_pil.filename).suffix
            except AttributeError:
                suffix = ".jpg"

        # set image file name and path
        slice_file_name = f"{output_file_name}_{slice_suffixes}{suffix}"

        # create coco image
        slice_width = slice_bbox[2] - slice_bbox[0]
        slice_height = slice_bbox[3] - slice_bbox[1]

        # create sliced image and append to sliced_image_result
        sliced_image = SlicedImage( image=image_pil_slice, starting_pixel=[slice_bbox[0], slice_bbox[1]] )
        sliced_image_result.add_sliced_image(sliced_image)

    # export slices if output directory is provided
    if output_file_name and output_dir:
        conc_exec = concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS)
        conc_exec.map(
            _export_single_slice,
            sliced_image_result.images,
            [output_dir] * len(sliced_image_result),
            sliced_image_result.filenames,
        )

    verboselog(
        "Num slices: " + str(n_ims) + " slice_height: " + str(slice_height) + " slice_width: " + str(slice_width)
    )

    return sliced_image_result


